TODO 

ADD TREMOR SCORE
ADD DICHOTOMIZATION
ADD MARKDOWN TO MAKE IT LOOK BETTER

THEN MISSING VALUES


In [230]:
#This cell is for defining various OPTIONS

import os 
#Input the directory where your database is located 
os.chdir('C:/Users/Briggstone/Documents/Master 2020/Parkinson_PPMI')

import pandas as pd 
import numpy as np

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)


In [231]:
#Reading the data table Screening demographics into a pandas dataframe
data = pd.read_csv('_Subject_Characteristics/Screening___Demographics.csv') 
print("Number of total subjects in original data table: ", data.shape[0])

#Discarding indviduals that do not have confirmed parkinson's, Everyone except APPRDX == 1.
data = data.loc[data.APPRDX == 1]
print("Number of subjects with parkinson's in data table:", data.shape[0])

#Selecting only PATNO column and reseting index (0-1-2 instead of 2-6-9). Index was weird due to dropping columns in the previous lines. 
pat_subset = data.loc[:, "PATNO"].reset_index(drop = True)

print(pat_subset.head())


Number of total subjects in original data table:  2200
Number of subjects with parkinson's in data table: 489
0    3400
1    3403
2    3406
3    3407
4    3150
Name: PATNO, dtype: int64


In [232]:
# Set this flag to 1 if using event_id for longitudinal joining or 0 if using infodt
long_flag = 1
LED = ["INFODT", "EVENT_ID"]


#Importing data tables from non-motor and selecting relevant columns
MOCA = pd.read_csv("Non-motor_Assessments/Montreal_Cognitive_Assessment__MoCA_.csv").loc[:,["PATNO", LED[long_flag], "MCATOT"]]

HPLT = pd.read_csv("Non-motor_Assessments/Hopkins_Verbal_Learning_Test.csv").loc[:,["PATNO", LED[long_flag], "DVT_DELAYED_RECALL"]]

BJLO = pd.read_csv("Non-motor_Assessments/Benton_Judgment_of_Line_Orientation.csv").loc[:,["PATNO", LED[long_flag], "JLO_TOTRAW"]]

LNSQ = pd.read_csv("Non-motor_Assessments/Letter_-_Number_Sequencing__PD_.csv").loc[:,["PATNO", LED[long_flag], "LNS_TOTRAW"]]

SEFL = pd.read_csv("Non-motor_Assessments/Semantic_Fluency.csv").loc[:,["PATNO", LED[long_flag], "VLTANIM", "VLTVEG", "VLTFRUIT"]]

REMQ = pd.read_csv("Non-motor_Assessments/REM_Sleep_Disorder_Questionnaire.csv").loc[:,["PATNO", LED[long_flag], "DRMVIVID", "DRMAGRAC", \
    "DRMNOCTB", "SLPLMBMV", "SLPINJUR", "DRMVERBL", "DRMFIGHT", "DRMUMV", "DRMOBJFL", "MVAWAKEN", "DRMREMEM", "SLPDSTRB", "STROKE", \
    "HETRA", "PARKISM", "RLS", "NARCLPSY", "DEPRS", "EPILEPSY", "BRNINFM", "CNSOTH"]]

GDSS = pd.read_csv("Non-motor_Assessments/Geriatric_Depression_Scale__Short_.csv").loc[:,["PATNO", LED[long_flag], "GDSSATIS", "GDSDROPD", \
    "GDSEMPTY", "GDSBORED", "GDSGSPIR", "GDSAFRAD", "GDSHAPPY", "GDSHLPLS", "GDSHOME", "GDSMEMRY", "GDSALIVE", "GDSWRTLS", "GDSENRGY", \
    "GDSHOPLS", "GDSBETER"]]

SIDT = pd.read_csv("Non-motor_Assessments/University_of_Pennsylvania_Smell_ID_Test.csv").loc[:,["PATNO", "UPSITBK1", "UPSITBK2", \
    "UPSITBK3", "UPSITBK4"]]

EPSS = pd.read_csv("Non-motor_Assessments/Epworth_Sleepiness_Scale.csv").loc[:,["PATNO", LED[long_flag], "ESS1", "ESS2", \
    "ESS3", "ESS4", "ESS5", "ESS6", "ESS7", "ESS8"]]

SCOP = pd.read_csv("Non-motor_Assessments/SCOPA-AUT.csv").loc[:,["PATNO", LED[long_flag], "SCAU1", "SCAU2", \
    "SCAU3", "SCAU4", "SCAU5", "SCAU6", "SCAU7", "SCAU8", "SCAU9", "SCAU10", "SCAU11", "SCAU12", "SCAU13", \
    "SCAU14", "SCAU15", "SCAU16", "SCAU17", "SCAU18", "SCAU19", "SCAU20", "SCAU21", "SCAU22", "SCAU23", "SCAU24", "SCAU25"]]



#Importing data tables from motor
MSU3 = pd.read_csv("Motor___MDS-UPDRS/MDS_UPDRS_Part_III.csv").loc[:,["PATNO", LED[long_flag], "NP3SPCH", "NP3FACXP", \
    "NP3RIGN", "NP3RIGRU", "NP3RIGRU", "PN3RIGRL", "NP3RIGLL", "NP3FTAPR", "NP3FTAPL", "NP3HMOVR", "NP3HMOVL", "NP3PRSPR", "NP3PRSPL", \
    "NP3TTAPR", "NP3TTAPL", "NP3TTAPL", "NP3LGAGL", "NP3RISNG", "NP3GAIT", "NP3FRZGT", "NP3PSTBL", "NP3POSTR", "NP3BRADY", "NP3PTRMR", \
    "NP3PTRML", "NP3KTRMR", "NP3KTRML", "NP3RTARU", "NP3RTALU", "NP3RTARL", "NP3RTALL", "NP3RTALJ", "NP3RTCON", "PD_MED_USE", "PAG_NAME"]]
MSU1 = pd.read_csv("Motor___MDS-UPDRS/MDS_UPDRS_Part_I.csv").loc[:,["PATNO", LED[long_flag], "NP1HALL", "NP1COG"]]



#Importing data tables from subject characteristics
FMHS = pd.read_csv("_Subject_Characteristics/Family_History__PD_.csv").loc[:,["PATNO", "BIOMOMPD", "BIODADPD", \
    "FULSIBPD", "HAFSIBPD", "MAGPARPD", "PAGPARPD", "MATAUPD", "PATAUPD", "KIDSPD"]]
SOEC = pd.read_csv("_Subject_Characteristics/Socio-Economics.csv").loc[:,["PATNO", "EDUCYRS"]]
SCDE = pd.read_csv("_Subject_Characteristics/Screening___Demographics.csv").loc[:,["PATNO", "PRJENRDT", "BIRTHDT"]]



In [233]:
#Additional preprocessing before merge

# SC to BL for MOCA as it has no Baseline data
MOCA.loc[MOCA["EVENT_ID"] == "SC", "EVENT_ID"] = "BL"

#Drop MSU3 data entries that are post drug-administering

MSU3 = MSU3.drop(MSU3[ MSU3['PAG_NAME'] == "NUPDRS3A" ].index)
MSU3 = MSU3.drop("PAG_NAME", axis = 1)

In [234]:
#Functions for calculating derived values for various data tables

def vlttot (df):
    component_vars = ["VLTANIM", "VLTVEG", "VLTFRUIT"]
    
    df['VLTTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    
def remqtot(df):
    
    component_vars = ["STROKE","HETRA", "PARKISM", "RLS", "NARCLPSY", "DEPRS", "EPILEPSY", "BRNINFM", "CNSOTH"]
    
    
    score = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    
    # 1 point if any of these component variables had a 1, else 0
    score = pd.Series(np.where(score >= 1, 1, 0))
    
    df.drop(component_vars, inplace = True, axis = 1)

    component_vars = ["DRMVIVID", "DRMAGRAC", "DRMNOCTB", "SLPLMBMV", "SLPINJUR", \
                      "DRMVERBL", "DRMFIGHT", "DRMUMV", "DRMOBJFL", "MVAWAKEN", "DRMREMEM", "SLPDSTRB"]
    

    score += df.loc[:, component_vars].sum(axis = 1, skipna = False)


    df['REMTOT'] = score
    df.drop(component_vars, inplace = True, axis = 1)
    
def gdsstot (df):
    
    component_vars = ["GDSSATIS", "GDSDROPD", \
    "GDSEMPTY", "GDSBORED", "GDSGSPIR", "GDSAFRAD", "GDSHAPPY", "GDSHLPLS", "GDSHOME", "GDSMEMRY", "GDSALIVE", "GDSWRTLS", "GDSENRGY", \
    "GDSHOPLS", "GDSBETER"]
    
    df['GDSSTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    
def sidttot (df):
    
    component_vars = ["UPSITBK1", "UPSITBK2", "UPSITBK3", "UPSITBK4"]
    
    df['SIDTTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    
def epsstot (df):
    
    component_vars = ["ESS1", "ESS2", \
    "ESS3", "ESS4", "ESS5", "ESS6", "ESS7", "ESS8"]
    
    df['EPSSTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    
def scoptot(df):
    #SCOP is not a simple sum and needs additional preprocessing to get total score
    # In SCAU1-21, 9 is converted to 3. In SCAU22-25, 9 is converted to 0
    for i in range(1,26):
        s = "SCAU" + str(i)
        points = 0
        if i < 22:
            points = 3
        df.loc[df[s] == 9, s] = points

    component_vars = ["SCAU1", "SCAU2", \
    "SCAU3", "SCAU4", "SCAU5", "SCAU6", "SCAU7", "SCAU8", "SCAU9", "SCAU10", "SCAU11", "SCAU12", "SCAU13", \
    "SCAU14", "SCAU15", "SCAU16", "SCAU17", "SCAU18", "SCAU19", "SCAU20", "SCAU21", "SCAU22", "SCAU23", "SCAU24", "SCAU25"]
    
    df['SCOPTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    

def msu3tot(df):

    component_vars = ["NP3SPCH", "NP3FACXP", \
    "NP3RIGN", "NP3RIGRU", "NP3RIGRU", "PN3RIGRL", "NP3RIGLL", "NP3FTAPR", "NP3FTAPL", "NP3HMOVR", "NP3HMOVL", "NP3PRSPR", "NP3PRSPL", \
    "NP3TTAPR", "NP3TTAPL", "NP3TTAPL", "NP3LGAGL", "NP3RISNG", "NP3GAIT", "NP3FRZGT", "NP3PSTBL", "NP3POSTR", "NP3BRADY", "NP3PTRMR", \
    "NP3PTRML", "NP3KTRMR", "NP3KTRML", "NP3RTARU", "NP3RTALU", "NP3RTARL", "NP3RTALL", "NP3RTALJ", "NP3RTCON"]
    
    df['MSU3TOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)


In [235]:
# First step of join: Merge non-longitudinal data
pat_subset = pd.DataFrame(pat_subset)
data = pat_subset.merge(FMHS, how = "inner", on = "PATNO")
data = data.merge(SOEC, how = "inner", on = "PATNO")
data = data.merge(SCDE, how = "inner", on = "PATNO")
data = data.merge(SIDT, how = "inner", on = "PATNO")


#Second step of join: Merge in longitudinal data

#This data table has duplicate entries for the same data, test is done before drug is administered and after
data = data.merge(MSU3, how = "inner", on ="PATNO")

data = data.merge(MSU1, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(MOCA, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(HPLT, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(BJLO, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(LNSQ, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(SEFL, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(GDSS, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(EPSS, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(SCOP, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(REMQ, how = "inner", on = ["PATNO", LED[long_flag]])


#Removing birthdate and project enrollment date and adding age as a function of these two columns

tempdata = data.loc[:, ["PRJENRDT", "BIRTHDT"]]
tempdata.PRJENRDT = tempdata.apply(lambda row_wise:int(row_wise["PRJENRDT"][-4:]), axis = 1)
tempdata.BIRTHDT = tempdata.apply(lambda row_wise: int(row_wise["BIRTHDT"]), axis = 1)
data ["AGE_BL"] = tempdata.PRJENRDT - tempdata.BIRTHDT
data = data.drop(["PRJENRDT", "BIRTHDT"], axis = 1)







In [236]:
import numpy as np
#Inspecting the merged data

#np.sort(data.columns.values)
print( "Data shape", data.shape)
print( "Number of unique PATNO:", data.PATNO.unique().size)

vlttot(data)
remqtot(data)
gdsstot(data)
sidttot(data)
epsstot(data)
scoptot(data)
msu3tot(data)

data.head(10)

Data shape (2342, 129)
Number of unique PATNO: 425


,PATNO,BIOMOMPD,BIODADPD,FULSIBPD,HAFSIBPD,MAGPARPD,PAGPARPD,MATAUPD,PATAUPD,KIDSPD,EDUCYRS,EVENT_ID,PD_MED_USE,NP1HALL,NP1COG,MCATOT,DVT_DELAYED_RECALL,JLO_TOTRAW,LNS_TOTRAW,AGE_BL,VLTTOT,REMTOT,GDSSTOT,SIDTTOT,EPSSTOT,SCOPTOT,MSU3TOT
0,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,BL,0,0.0,0.0,24.0,54.0,13.0,10.0,39,63.0,10.0,7.0,30.0,12.0,NaN,26.0
1,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V04,3,0.0,1.0,29.0,54.0,12.0,12.0,39,49.0,7.0,11.0,30.0,9.0,NaN,55.0
2,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V06,2,0.0,1.0,30.0,44.0,12.0,10.0,39,53.0,4.0,8.0,30.0,8.0,NaN,37.0
3,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V12,4,0.0,1.0,29.0,54.0,12.0,2.0,39,49.0,4.0,8.0,30.0,13.0,NaN,25.0
4,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V08,6,0.0,2.0,30.0,54.0,12.0,12.0,39,59.0,8.0,9.0,30.0,15.0,NaN,41.0
5,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V10,7,0.0,2.0,26.0,39.0,12.0,7.0,39,50.0,5.0,7.0,30.0,15.0,NaN,54.0
6,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V13,7,0.0,2.0,28.0,49.0,14.0,8.0,39,52.0,1.0,9.0,30.0,18.0,NaN,35.0
7,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V14,7,1.0,1.0,28.0,49.0,13.0,9.0,39,48.0,2.0,10.0,30.0,20.0,NaN,43.0
8,3400,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,V15,7,1.0,0.0,27.0,49.0,13.0,9.0,39,55.0,1.0,8.0,30.0,16.0,NaN,41.0
9,3403,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,18.0,BL,0,0.0,0.0,25.0,52.0,13.0,10.0,69,38.0,5.0,4.0,9.0,8.0,NaN,39.0


In [237]:
#Exploring missing values

null_columns=data.columns[data.isnull().any()]
print(data[null_columns].isnull().sum())

print("\nUnique patients entries with BASELINE OR SCREENING")
tempdata = data.loc[(data.EVENT_ID == 'BL') | (data.EVENT_ID == "SC")]
print(tempdata.PATNO.unique().size)

print("\nEntries with missing MCATOT")
print(data[data['MCATOT'].isnull()][["PATNO", LED[long_flag], "MCATOT"]].PATNO.unique())

print("\nEntries with missing DVT_DELAYED_RECALL")
print(data[data['DVT_DELAYED_RECALL'].isnull()][["PATNO", LED[long_flag], "DVT_DELAYED_RECALL"]].PATNO.unique())

print("\nEntries with missing JLO_TOTRAW")
print(data[data['JLO_TOTRAW'].isnull()][["PATNO", LED[long_flag], "JLO_TOTRAW"]].PATNO.unique())

print("\nEntries with missing BIOMOMPD")
print(data[data['BIOMOMPD'].isnull()][["PATNO", LED[long_flag], "BIOMOMPD"]].PATNO.unique())

print("\nEntries with missing BIODADPD")
print(data[data['BIODADPD'].isnull()][["PATNO", LED[long_flag], "BIODADPD"]].PATNO.unique())

print("\nEntries with missing FULSIBPD")
print(data[data['FULSIBPD'].isnull()][["PATNO", LED[long_flag], "FULSIBPD"]].PATNO.unique().size)

print("\nEntries with missing HAFSIBPD")
print(data[data['HAFSIBPD'].isnull()][["PATNO", LED[long_flag], "HAFSIBPD"]].PATNO.unique().size)

print("\nEntries with missing MAGPARPD")
print(data[data['MAGPARPD'].isnull()][["PATNO", LED[long_flag], "MAGPARPD"]].PATNO.unique().size)

print("\nEntries with missing PAGPARPD")
print(data[data['PAGPARPD'].isnull()][["PATNO", LED[long_flag], "PAGPARPD"]].PATNO.unique().size)

print("\nEntries with missing MATAUPD")
print(data[data['MATAUPD'].isnull()][["PATNO", LED[long_flag], "MATAUPD"]].PATNO.unique().size)

print("\nEntries with missing PATAUPD")
print(data[data['PATAUPD'].isnull()][["PATNO", LED[long_flag], "PATAUPD"]].PATNO.unique().size)

print("\nEntries with missing KIDSPD")
print(data[data['KIDSPD'].isnull()][["PATNO", LED[long_flag], "KIDSPD"]].PATNO.unique().size)

print("\nEntries with missing SIDTTOT")
print(data[data['SIDTTOT'].isnull()][["PATNO", LED[long_flag], "SIDTTOT"]].PATNO.unique())

print("\nEntries with missing MSU3TOT")
print(data[data['MSU3TOT'].isnull()][["PATNO", LED[long_flag], "MSU3TOT"]].PATNO.unique())

print("\nEntries with missing GDSSTOT")
print(data[data['GDSSTOT'].isnull()][["PATNO", LED[long_flag], "GDSSTOT"]].PATNO.unique())

print("\nEntries with missing EPSSTOT")
print(data[data['EPSSTOT'].isnull()][["PATNO", LED[long_flag], "EPSSTOT"]].PATNO.unique())

print("\nEntries with missing SCOPTOT")
print(data[data['SCOPTOT'].isnull()][["PATNO", LED[long_flag], "SCOPTOT"]].PATNO.unique().size)

BIOMOMPD                16
BIODADPD                16
FULSIBPD               145
HAFSIBPD              1573
MAGPARPD                22
PAGPARPD                24
MATAUPD                141
PATAUPD                189
KIDSPD                 272
MCATOT                   2
DVT_DELAYED_RECALL       3
JLO_TOTRAW               3
REMTOT                   8
GDSSTOT                  4
SIDTTOT                  1
EPSSTOT                  3
SCOPTOT               2339
MSU3TOT                 11
dtype: int64

Unique patients entries with BASELINE OR SCREENING
424

Entries with missing MCATOT
[3076]

Entries with missing DVT_DELAYED_RECALL
[3001 3061 3028]

Entries with missing JLO_TOTRAW
[3116 3076]

Entries with missing BIOMOMPD
[3604 3770 4073]

Entries with missing BIODADPD
[3604 3770 3834]

Entries with missing FULSIBPD
28

Entries with missing HAFSIBPD
283

Entries with missing MAGPARPD
4

Entries with missing PAGPARPD
5

Entries with missing MATAUPD
29

Entries with missing PATAUPD
35

Entries 